In [ ]:
!pip install torch

In [ ]:
!pip install transformers==4.24.0

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer

from google.colab import drive
drive.mount('/content/drive')

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("device:", device)

In [ ]:
# 토크나이저
MODEL_NAME = "beomi/KcELECTRA-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
#load model
model = torch.load('/content/drive/MyDrive/colab_data/kcelectra_trained_model.pt',map_location=device)
model.eval()

In [ ]:
def predict_sent(sent,model,tokenizer):
    
    # 입력된 문장 토크나이징
    tokenized_sent = tokenizer(
        sent,
        return_tensors="pt",
        truncation=True,
        add_special_tokens=True,
        max_length=128
    )

    # 모델이 위치한 GPU로 이동 
    tokenized_sent.to(device)

    # 예측
    with torch.no_grad():
        outputs = model(
            input_ids=tokenized_sent["input_ids"],
            attention_mask=tokenized_sent["attention_mask"],
            token_type_ids=tokenized_sent["token_type_ids"]
            )

    # 결과 return
    logits = outputs[0]
    
    logits = logits.detach().cpu()
    result = logits.argmax(-1)
    if result == 0:
        result = " >> 악성댓글 👿"
    elif result == 1:
        result = " >> 정상댓글 😀"
        
    return result

In [ ]:
sent = input()

In [ ]:
predict_sent(sent,model,tokenizer)